In [1]:
from bs4 import BeautifulSoup
import requests
import re
import fractions

In [2]:
"""

web scraper

"""

def get_numbers(lst):
    new_list = [int(x.split(':')[1]) if ':' in x else -1 for x in lst]
    return new_list

class RecipeFetcher:

    search_base_url = 'https://www.allrecipes.com/search/results/?wt=%s&sort=re'

    def search_recipes(self, keywords): 
        search_url = self.search_base_url %(keywords.replace(' ','+'))

        page_html = requests.get(search_url)
        page_graph = BeautifulSoup(page_html.content)

        return [recipe.a['href'] for recipe in\
               page_graph.find_all('div', {'class':'grid-card-image-container'})]

    def scrape_recipe(self, recipe_url):
        results = {}

        page_html = requests.get(recipe_url)
        print(recipe_url)
        page_graph = BeautifulSoup(page_html.content)

        results['ingredients'] = [ingredient.text for ingredient in\
                                  page_graph.find_all('span', {'itemprop':'recipeIngredient'})]

        results['directions'] = [direction.text.strip() for direction in\
                                 page_graph.find_all('span', {'class':'recipe-directions__list--item'})
                                 if direction.text.strip()]

        results['nutrition'] = self.scrape_nutrition_facts(recipe_url)
        
        results['calories_and_servings'] = self.scrape_calories_servings(recipe_url)

        return results
    
    def scrape_nutrition_facts(self, recipe_url):
        results = []

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for nutrient_row in page_graph.find_all('div', {'class': 'nutrition-row'}):
            
            lst = nutrient_row.text.split(':')
            amount_lst = lst[1]
            name = lst[0].replace('\n', '')
            
            amount = amount_lst.split('\n')
            amount = [x.replace(' ', '') for x in amount[:2]]
            
            nutrient[name] = amount
            
        return nutrient
    
    def scrape_calories_servings(self, recipe_url):
        """
        returns [servings per recipe, amt per serving, calories]
        """

        nutrition_facts_url = '%s/fullrecipenutrition' %(recipe_url)

        page_html = requests.get(nutrition_facts_url)
        page_graph = BeautifulSoup(page_html.content)

        r = re.compile("([0-9]*\.?[0-9]*)([a-zA-Z]+)")
        
        nutrient = {}

        for row in page_graph.find_all('div', {'class': 'nutrition-top light-underline'}):
            lst = row.text.split('\n')
            lst = list(filter(lambda a: a != '\r', lst))
            
            calories = [x.lstrip() for x in lst]
            calories.pop()
            info = get_numbers(calories)
            
            return info

In [57]:
rf = RecipeFetcher()
fluffy_pancakes = rf.search_recipes('lobster')[0]
res = rf.scrape_recipe(fluffy_pancakes)

https://www.allrecipes.com/recipe/36996/grilled-rock-lobster-tails/


In [58]:
ingredients = res['ingredients']
ingredients

['1 tablespoon lemon juice',
 '1/2 cup olive oil',
 '1 teaspoon salt',
 '1 teaspoon paprika',
 '1/8 teaspoon white pepper',
 '1/8 teaspoon garlic powder',
 '2 (10 ounce) rock lobster tails']

In [59]:
"""
- replace ground meat with ground tofu
- replace meat with tofu
- replace hamburger meat with veggie paddy

"""

'\n- replace ground meat with ground tofu\n- replace meat with tofu\n- replace hamburger meat with veggie paddy\n\n'

In [60]:
def look_for_meat(ingredients):
    ground_meat_lst = ['ground beef', 'ground chicken', 'ground meat', 'ground turkey', 'ground lamb', 'ground pork', 'ground bison']
    meat_lst = ['chicken', 'steak', 'beef', 'lamb', 'bacon', 'pork', 'duck', 'bison', 'rabbit', 'cow', 'turkey']
    sandwich_meat_lst = ['hamburger', 'cheeseburger', 'sloppy joe']
    seafood_lst = ['seafood', 'salmon', 'cod', 'fish', 'halibut', 'shellfish', 'crab', 'lobster', 'shrimp', 'prawn', 'scallop']
    
    ground = False
    meat = False
    sandwich = False
    seafood = False
    
    for ingredient in ingredients:
        if any(x in ingredient.lower() for x in ground_meat_lst):
            ground = True
            replace_meat_lst = [ground, meat, sandwich, seafood]
            return ingredient, replace_meat_lst
        if any(x in ingredient.lower() for x in meat_lst):
            meat = True
            replace_meat_lst = [ground, meat, sandwich, seafood]
            return ingredient, replace_meat_lst
        if any(x in ingredient.lower() for x in sandwich_meat_lst):
            sandwich = True
            replace_meat_lst = [ground, meat, sandwich, seafood]
            return ingredient, replace_meat_lst
        if any(x in ingredient.lower() for x in seafood_lst):
            seafood = True
            replace_meat_lst = [ground, meat, sandwich, seafood]
            return ingredient, replace_meat_lst

In [61]:
p, pp = look_for_meat(ingredients)

In [62]:
p

'2 (10 ounce) rock lobster tails'

In [63]:
pp

[False, False, False, True]

In [64]:
def replace_meat(ingredients, meat, type_of_meat_lst):
    new_meat = meat.replace("(", "")
    new_meat = new_meat.replace(")", "")
    new_meat = new_meat.replace("/", " ")
    meat_lst = new_meat.split()
    
    print(meat_lst)
    
    original_quantity = []
    ingredient_name = ''

    # split ingredient into tokens to separate the 'quantity' from 'ingredient_name'
    for token in meat_lst:            
        if any([str(digit) in token for digit in range(10)]) and not any([char in token for char in ['(', ')']]):
            fraction_obj = sum(map(fractions.Fraction, token.split()))
            as_float = int(fraction_obj)
            original_quantity.append(as_float)
        else:
            ingredient_name = ingredient_name + ' ' + token

    # multiply amount by 'multiplier'
    new_quantity = original_quantity
    
    print("new: {}".format(new_quantity))
    
    if len(new_quantity) > 1:
        num = new_quantity[-1]
    else:
        num = new_quantity[0]
    
    print(num)
    
    
    
    
    
    
    
    
    
    #quantity = ""
    
    #print(meat_lst)
    #print([int(s) for s in meat.split() if s.isdigit()])

    # split ingredient into tokens to separate the 'quantity' from 'ingredient_name'
    """
    
    for word in meat:            
        if any([str(digit) in word for digit in range(10)]) and not any([char in word for char in ['(', ')']]):
            print(word)
            quantity += word
            #break
    """
            
    replace_idx = meat_lst.index(str(num)) + 2

    # ground meat replacement
    if type_of_meat_lst[0]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "ground tofu")
        print(veggie)
       
    # meat replacement
    if type_of_meat_lst[1]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "tofu")
        print(veggie)
        
    # sandwich meat replacement
    if type_of_meat_lst[2]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "impossible burger")
        print(veggie)
        
    # seafood replacement
    if type_of_meat_lst[3]:
        replace_term = meat_lst[replace_idx: ]
        s = " "
        term = s.join(replace_term)
        veggie = meat.replace(term, "tofuna fysh")
        print(veggie)

In [65]:
replace_meat(ingredients, p, pp)

['2', '10', 'ounce', 'rock', 'lobster', 'tails']
new: [2, 10]
10
2 (10 ounce) tofuna fysh
